In [1]:
import timbermafia
import logging
import sys
from timbermafia.formatters import *
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

In [2]:
timbermafia.configure_root_logger(stream=sys.stdout)
log = logging.getLogger('testing')

def test_log():
    log.debug('test from func')

In [3]:
log.hinfo('Timbermafia testing')
log.info(lorem)
log.info('This looks like a filename: /tmp/a_file.db')
log.info('This looks like a url: https://someaddress.com')

log.debug('a debug message')
log.warning('a warning message')
log.error('found error we can recover from')

------------------------------------------------------------------------------------------------------------------------
--- Logging error ---
Traceback (most recent call last):
  File "/home/sogilvy/repos/timbermafia/timbermafia/rainbow.py", line 42, in emit
    message = self.format(record)
  File "/home/sogilvy/repos/timbermafia/timbermafia/rainbow.py", line 79, in format
    message = logging.StreamHandler.format(self, record)
  File "/home/sogilvy/miniconda3/envs/timbermafia/lib/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/home/sogilvy/repos/timbermafia/timbermafia/formatters.py", line 157, in format
    s_list = textwrap.wrap(print_statement, self.message_padding - 3,
  File "/home/sogilvy/repos/timbermafia/timbermafia/formatters.py", line 33, in message_padding
    return self.columns - 30 - self.time_padding
TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'
Call stack:
  File "/home/sogilvy/miniconda3/envs/timbermaf

['name', 'funcName', 'asctime']


--- Logging error ---
Traceback (most recent call last):
  File "/home/sogilvy/repos/timbermafia/timbermafia/rainbow.py", line 42, in emit
    message = self.format(record)
  File "/home/sogilvy/repos/timbermafia/timbermafia/rainbow.py", line 79, in format
    message = logging.StreamHandler.format(self, record)
  File "/home/sogilvy/miniconda3/envs/timbermafia/lib/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/home/sogilvy/repos/timbermafia/timbermafia/formatters.py", line 157, in format
    s_list = textwrap.wrap(print_statement, self.message_padding - 3,
  File "/home/sogilvy/repos/timbermafia/timbermafia/formatters.py", line 33, in message_padding
    return self.columns - 30 - self.time_padding
TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'
Call stack:
  File "/home/sogilvy/miniconda3/envs/timbermafia/lib/python3.8/runpy.py", line 193, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home

In [4]:
f = log.parent.handlers[0].formatter

In [5]:
f._fmt

'{asctime} | {name}.{funcName} | {message}'

In [6]:
f.separator
    


'|'

In [7]:
sections = [s.strip() for s in f._fmt.split(f.separator)]

In [8]:
l = logging.LogRecord('test', logging.INFO, 'testpath', 12, 'this is a very very very obtusely ridiculously fucking long and inconvenient kind of shit message',[], False, func='my_func_that_has_a_totally_ridiculously_long_name')
# f.format(l)
s = super(TMFormatter, f).format(l)
s

'05:17:33 | test.my_func_that_has_a_totally_ridiculously_long_name | this is a very very very obtusely ridiculously fucking long and inconvenient kind of shit message'

In [9]:
chunks = [chunk.strip() for chunk in s.split(f.separator)]
time_length = None
time = None
for section, message in zip(sections, chunks):
    print(f'{section}: {message}')
    # Length of time string will be uniform so adaptively read it.
    if 'asctime' in section:
        time = message
        time_length = len(message) + 2  # add the space and divisor
time_length

{asctime}: 05:17:33
{name}.{funcName}: test.my_func_that_has_a_totally_ridiculously_long_name
{message}: this is a very very very obtusely ridiculously fucking long and inconvenient kind of shit message


10

In [10]:
sections
chunks

['{asctime}', '{name}.{funcName}', '{message}']

['05:17:33',
 'test.my_func_that_has_a_totally_ridiculously_long_name',
 'this is a very very very obtusely ridiculously fucking long and inconvenient kind of shit message']

In [11]:
f.padding_dict

{'name': 12,
 'funcName': 13,
 'module': 25,
 'pathname': 40,
 'lineNo': 4,
 'thread': 4,
 'threadName': 10,
 'asctime': 8,
 'message': 81}

In [12]:
import textwrap
def pad(header, content):
    
    # Find the fields present
    fields = [s for s in f.padding_dict if s in header]
    
    # Add the padding for each field
    padding = 0
    for field in fields:
        padding += f.padding_dict[field]
    
    # textwrap the results
    content_list = textwrap.wrap(content, padding, break_long_words=True)
#     print(fields, padding)

    return content_list, padding

    


In [13]:
contents = {}
max_lines = 0
for i, (s, c) in enumerate(zip(sections, chunks)):
    s_list, padding = pad(s, c)
    n_lines = len(s_list)
    if max_lines < n_lines:
        max_lines = n_lines
    contents[i] = {'line_array': s_list, 'padding': padding}
    

In [14]:
contents

{0: {'line_array': ['05:17:33'], 'padding': 8},
 1: {'line_array': ['test.my_func_that_has_a_t',
   'otally_ridiculously_long_',
   'name'],
  'padding': 25},
 2: {'line_array': ['this is a very very very obtusely ridiculously fucking long and inconvenient kind',
   'of shit message'],
  'padding': 81}}

In [15]:
max_lines

3

In [16]:
contents[0]['line_array'][0]

'05:17:33'

In [17]:
complete_s = ''
for i in range(max_lines):
    for key, results in contents.items():
        try:
            complete_s += results['line_array'][i].ljust(results['padding'])
        except IndexError:
            complete_s += ' '* results['padding']
        complete_s += f' {f.separator} '
    complete_s += '\n'
    
print(complete_s)

05:17:33 | test.my_func_that_has_a_t | this is a very very very obtusely ridiculously fucking long and inconvenient kind | 
         | otally_ridiculously_long_ | of shit message                                                                   | 
         | name                      |                                                                                   | 

